# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»
#### Предобработка данных

## Описание данных:
* 'status' — статус продажи;
* 'private pool' и 'PrivatePool' — наличие собственного бассейна;
* 'propertyType' — тип объекта недвижимости;
* 'street' — адрес объекта;
* 'baths' — количество ванных комнат;
* 'homeFacts' — сведения о строительстве объекта (содержит несколько
типов сведений, влияющих на оценку объекта);
* 'fireplace' — наличие камина;
* 'city' — город;
* 'schools' — сведения о школах в районе;
* 'sqft' — площадь в футах;
* 'zipcode' — почтовый индекс;
* 'beds' — количество спален;
* 'state' — штат;
* 'stories' — количество этажей;
* 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система
мультилистинга);
* target' — цена объекта недвижимости (целевой признак, который
необходимо спрогнозировать).

In [1]:
# Загрузка необходимых библиотек
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim
from fake_useragent import UserAgent
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.exc import GeocoderServiceError

# Первичный осмотр данных
Посмотрим на выгруженные данные.

In [2]:
df = pd.read_csv('data/data.csv')
display(df.head())
df.info()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

Сразу подчистим явные дубликаты

In [3]:
# сохраняем количество строк до удаления дубликатов
rows_before = df.shape[0]
#удалим дубликаты
df = df.drop_duplicates(ignore_index=True)
# сохраняем количество строк после удаления дубликатов
rows_after = df.shape[0]
# считаем количество удаленных дубликатов
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 50 дубликатов


## Посмотрим на пропуски

In [4]:
missing_data = df.isnull().sum()

# Выводим столбцы, которые содержат пропуски и их количество (в процентах)
for column_name, missing_count in missing_data.items():
    if missing_count > 0:
        percentage = (missing_count / len(df)) * 100
        print(f'{column_name}: {missing_count} пропусков ({percentage:.2f}% от общего числа данных в столбце)')

status: 39917 пропусков (10.58% от общего числа данных в столбце)
private pool: 372954 пропусков (98.89% от общего числа данных в столбце)
propertyType: 34733 пропусков (9.21% от общего числа данных в столбце)
street: 2 пропусков (0.00% от общего числа данных в столбце)
baths: 106308 пропусков (28.19% от общего числа данных в столбце)
fireplace: 274024 пропусков (72.66% от общего числа данных в столбце)
city: 34 пропусков (0.01% от общего числа данных в столбце)
sqft: 40550 пропусков (10.75% от общего числа данных в столбце)
beds: 91254 пропусков (24.20% от общего числа данных в столбце)
stories: 150674 пропусков (39.95% от общего числа данных в столбце)
mls-id: 352193 пропусков (93.39% от общего числа данных в столбце)
PrivatePool: 336825 пропусков (89.31% от общего числа данных в столбце)
MlsId: 66869 пропусков (17.73% от общего числа данных в столбце)
target: 2480 пропусков (0.66% от общего числа данных в столбце)


In [5]:
# посмотрим что содержится в private pool и PrivatePool
print(df['private pool'].unique())
print(df['PrivatePool'].unique())

[nan 'Yes']
[nan 'yes' 'Yes']


#### Поскольку 'private pool' и 'PrivatePool' содержат дублирующую информацию то объеденим их так же отметим поскольку у нас всего 3 значения то вероятнее всего пропуск свидетельствует о отсутствии бассейна закодируем его булевыми переменными, а так же удалим 'mls-id' и 'MlsId' поскольку это признаки существуют для идентификации объекта в общих базах риэлторов для наших задач он не требуется

In [6]:
# Объединяем столбцы
df['pool_encoded'] = df['private pool'].combine_first(df['PrivatePool'])
# Удаляем столбец PrivatePool
df = df.drop(['PrivatePool', 'private pool', 'MlsId', 'mls-id'], axis=1)
# Заменить NaN на False
df['pool_encoded'] = df['pool_encoded'].fillna(False)
# Заменить 'yes' и 'Yes' на True
df['pool_encoded'] = df['pool_encoded'].replace(['yes', 'Yes'], True)
# Выводим результат
print("Уникальные значения 'pool_encoded':", df['pool_encoded'].unique())
display(df)


Уникальные значения 'pool_encoded': [False  True]


,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded
0,Active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,"$418,000",False
1,for sale,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,"$310,000",False
2,for sale,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,"$2,895,000",True
3,for sale,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,"$2,395,000",False
4,for sale,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377130,NaN,Single Family,20800 NE 23rd Ave,6.0,"{'atAGlanceFacts': [{'factValue': '1990', 'fac...",NaN,Miami,"[{'rating': ['10/10', '5/10'], 'data': {'Dista...","4,017",33180,5,FL,0.0,"$1,249,000",True
377131,for sale,condo,3530 N Lake Shore Dr #4B,3 Baths,"{'atAGlanceFacts': [{'factValue': '1924', 'fac...",NaN,Chicago,"[{'rating': ['1/10', '5/10', '7/10'], 'data': ...","2,000 sqft",60657,3 Beds,IL,9.0,"$674,999",False
377132,for sale,single-family home,15509 Linden Blvd,3 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,Jamaica,"[{'rating': ['5/10', '4/10'], 'data': {'Distan...","1,152 sqft",11434,3 Beds,NY,2,"$528,000",False
377133,NaN,NaN,7810 Pereida St,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Houston,"[{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...",NaN,77028,"8,479 sqft",TX,NaN,"$34,500",False


In [7]:
# сохраняем количество строк до удаления дубликатов
rows_before = df.shape[0]
#удалим дубликаты
df = df.drop_duplicates(ignore_index=True)
# сохраняем количество строк после удаления дубликатов
rows_after = df.shape[0]
# считаем количество удаленных дубликатов
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 91 дубликатов


### Обработаем признак status

In [8]:
missing_values_count = df.status.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.status.nunique()}\n"
      f"Уникальные значения: {list(df.status.unique())}")

Количество пропущенных значений: 39916
Процент пропущенных значений: 10.59%
Количество уникальных значений: 159
Уникальные значения: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction', ' / auction', 'Under Contract', 'Under Contract   Showing', 'Pre-foreclosure', 'Under Contract Backups', 'foreclosure', 'Active Under Contract', 'Foreclosed', 'Option Pending', 'Under Contract Show', 'for rent', 'Auction', 'A Active', 'Contingent', 'Pending   Continue To Show', 'Price Change', 'Back on Market', 'Active Option', 'Foreclosure', 'recently sold', 'Coming soon: Nov 21.', 'Contingent Finance And Inspection', 'Coming soon: Dec 4.', 'P Pending Sale', 'Coming soon: Nov 23.', 'Active With Contingencies', 'Pending Ab', 'Pf', 'Contingent Show', 'Contract P', 'Contingent Take Backup', 'Apartment for rent', 'Backup Contract', 'Option Contract', 'Pending Continue To Show', 'pending', 'Pending Inspection', 'Active Option Co

#### Разделим статусы на категории а пропуски заменим на other и запишем их

In [9]:
categories = {
    "For Sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"
    ],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"
    ],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"
    ],
    "Foreclosure": [
        "Pre-foreclosure", "Pre-foreclosure / auction", " / auction", "Foreclosed", "foreclosure", "Foreclosure"
    ],
    "Under Contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Due Diligence Period", "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"
    ],
    "For Rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction"],
    "Recently Sold": ["recently sold"],
    "Price Change": ["Price Change"],
    "Back on Market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing Extended": ["Listing Extended"],
    "Coming Soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."
    ],
}

def categorize_status(status_value):
    for category, values in categories.items():
        if status_value in values:
            return category
    return "Other"

# заменим значения на категории
df['status'] = df['status'].apply(categorize_status)



In [10]:
# выполним проверку
missing_values_count = df.status.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.status.nunique()}\n"
      f"Уникальные значения: {list(df.status.unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 15
Уникальные значения: ['Active', 'For Sale', 'Other', 'Pending', 'Foreclosure', 'Under Contract', 'For Rent', 'Auction', 'Contingent', 'Price Change', 'Back on Market', 'Recently Sold', 'Coming Soon', 'Closed', 'Listing Extended']


In [11]:
# сохраняем количество строк до удаления дубликатов
rows_before = df.shape[0]
#удалим дубликаты
df = df.drop_duplicates(ignore_index=True)
# сохраняем количество строк после удаления дубликатов
rows_after = df.shape[0]
# считаем количество удаленных дубликатов
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 0 дубликатов


#### Посмотрим на признак propertyType

In [12]:
# выполним проверку
missing_values_count = df.propertyType.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.propertyType.nunique()}\n"
      f"Уникальные значения: {list(df.propertyType.unique())}")

Количество пропущенных значений: 34725
Процент пропущенных значений: 9.21%
Количество уникальных значений: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story', 'Townhouse', 'multi-family', 'Penthouse, Split-Level', 'Multi-Family Home', 'Condo', 'condo', 'Land', 'Condo/Townhome/Row Home/Co-Op', ' ', 'Detached, Two Story', '1 Story, Ranch', 'Other Style', 'Colonial', 'Transitional', 'High Rise', 'mobile/manufactured', 'Tri-Level, Northwestern Contemporary', 'Detached, One Story', 'Craftsman', 'Single Detached, French', '1 Story, Traditional', 'Single Detached, Traditional', 'Federal', 'Multi Family', 'apartment', 'Traditional', 'Custom', 'Cooperative', 'Contemporary/Modern, Traditional', 'Cape Cod, Contemporary, Florida, Key West', 'Single Detached', 'Mobile / Manufactured', 'Contemporary/Modern', 'Miscellaneous', 'Mfd/Mobile Home', 'Bungalow', '1 Story', 'Spanish/Mediterranean', 'Co

In [13]:
# переведем все в строчные
df.propertyType = df.propertyType.str.lower()
# выполним проверку
missing_values_count = df.propertyType.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.propertyType.nunique()}\n"
      f"Уникальные значения: {list(df.propertyType.unique())}")

Количество пропущенных значений: 34725
Процент пропущенных значений: 9.21%
Количество уникальных значений: 1272
Уникальные значения: ['single family home', 'single-family home', 'lot/land', 'townhouse', 'florida', nan, 'single family', 'coop', 'english', '2 story', 'multi-family', 'penthouse, split-level', 'multi-family home', 'condo', 'land', 'condo/townhome/row home/co-op', ' ', 'detached, two story', '1 story, ranch', 'other style', 'colonial', 'transitional', 'high rise', 'mobile/manufactured', 'tri-level, northwestern contemporary', 'detached, one story', 'craftsman', 'single detached, french', '1 story, traditional', 'single detached, traditional', 'federal', 'multi family', 'apartment', 'traditional', 'custom', 'cooperative', 'contemporary/modern, traditional', 'cape cod, contemporary, florida, key west', 'single detached', 'mobile / manufactured', 'contemporary/modern', 'miscellaneous', 'mfd/mobile home', 'bungalow', '1 story', 'spanish/mediterranean', 'contemporary', 'multi-le

как видно из структуры тип записан в 1 значении отделенным запятой

In [14]:
# приведем разное написание к 1 виду а так же заменим yes и unknown на Other
df.propertyType = df.propertyType \
.str.replace('single-family home','single family') \
.str.replace('single family home','single family') \
.str.replace('yes','Other') \
.str.replace('unknown','Other')
# Создадим признак Type и запишем в него объединенные по 1 слову данные 
df['Type'] = df['propertyType'].str.split(',').str[0]

In [15]:
# выполним проверку
missing_values_count = df.Type.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.Type.nunique()}\n"
      f"Уникальные значения: {list(df.Type.unique())}")

Количество пропущенных значений: 34725
Процент пропущенных значений: 9.21%
Количество уникальных значений: 256
Уникальные значения: ['single family', 'lot/land', 'townhouse', 'florida', nan, 'coop', 'english', '2 story', 'multi-family', 'penthouse', 'multi-family home', 'condo', 'land', 'condo/townhome/row home/co-op', ' ', 'detached', '1 story', 'other style', 'colonial', 'transitional', 'high rise', 'mobile/manufactured', 'tri-level', 'craftsman', 'single detached', 'federal', 'multi family', 'apartment', 'traditional', 'custom', 'cooperative', 'contemporary/modern', 'cape cod', 'mobile / manufactured', 'miscellaneous', 'mfd/mobile home', 'bungalow', 'spanish/mediterranean', 'contemporary', 'multi-level', 'condo/unit', '2 stories', 'ranch', 'low-rise (1-3 stories)', 'rancher', 'urban contemporary', 'two story', 'garden home', 'farms/ranches', 'a-frame', 'attached or 1/2 duplex', 'one story', 'georgian', 'dwelling with rental', 'victorian', 'cluster home', 'manufactured house', 'other

In [16]:
synonyms_mapping = {
    'single_family_home': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached', 'singlefamilyresidence', 'single wide', 'single-wide mobile with land',
        'two story', 'one story', 'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'],
    'multi_family_home': [
        'multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex', 'multi_level', 'multi family', '2 unit condo', '2-story'],
    'condo': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise'],
    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'],
    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'Flats', 'studio'],
    'land': [
        'lot/land', 'land'],
    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home', 'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story', 'mobile manu - double wide'],
    'miscellaneous': [
        'miscellaneous'],
    'ranch': [
        'ranch', 'one story'],
    'modern': [
        'contemporary', 'contemporary/modern', "modern", 'mid-century modern', 'modern farmhouse', 'modernist'],
    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older', 'historic vintage', 'historic'],
    'other': [
        'Other', 'english', 'urban contemporary', 'other style', 'florida', 'farms/ranches', 'carriage house', 'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor', 'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house', 'farm house', 'hi ranch', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', 'victorian', '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam', 'manuf/mobile', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home', '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement', 'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor', 'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch','residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio', 'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii', 'coastal modern', 'coastal two story', 'mountain contemporary', 'key west', 'high ranch', 'end unit']
}


def replace_synonyms(value):
    for key, synonym_values in synonyms_mapping.items():
        if value in synonym_values:
            return key
    return "other"

# Замена синонимов
df['Type'] = df['Type'].apply(replace_synonyms)



In [17]:
# выполним проверку
missing_values_count = df.Type.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.Type.nunique()}\n"
      f"Уникальные значения: {list(df.Type.unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 12
Уникальные значения: ['single_family_home', 'land', 'townhouse', 'other', 'condo', 'multi_family_home', 'mobile_home', 'apartment', 'modern', 'miscellaneous', 'ranch', 'historical']


In [18]:
# теперь propertyType можно удалить
df.drop('propertyType', axis=1, inplace=True)


Обработаем Признак street

In [19]:
# выполним проверку
missing_values_count = df.street.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.street.nunique()}")

Количество пропущенных значений: 2
Процент пропущенных значений: 0.00%
Количество уникальных значений: 337076


In [20]:
# удаляем строки, где есть пропуски в столбце street
df = df.dropna(subset=['street'])

#### C помощью библиотеки geopy получим кординаты расположения

In [21]:
#user_agent = UserAgent().random
#geolocator = Nominatim(user_agent=user_agent, timeout=10)

#def get_coordinates(address, retry_count=3):
#    if retry_count <= 0:
#       return None

#    try:
#        location = geolocator.geocode(address)
#        if location:
#            return (location.latitude, location.longitude)
#        else:
#            return None
#    except (GeocoderTimedOut, GeocoderUnavailable, GeocoderServiceError):
#        return get_coordinates(address, retry_count - 1)

#df['coordinates'] = df['street'].apply(get_coordinates)

#Поскольку данный способ очень продолжительный по времени пришлось от него отказаться

Оставим пока этот признак так как не понятно как его можно использовать но информация из него может пригодиться в дальнейшем

Обработаем признак baths

In [22]:
# выполним проверку
missing_values_count = df.baths.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.baths.nunique()}\n"
      f"Уникальные значения: {list(df.baths.unique())}")

Количество пропущенных значений: 106260
Процент пропущенных значений: 28.18%
Количество уникальных значений: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5', '1,000', '7 Baths', '2.0', '3.0', 'Bathrooms: 1', '4.0', '2.1 Baths', '2.5 Baths', '1', 'Bathrooms: 3', '4.5', '6 Baths', 'Bathrooms: 4', '3 ba', '5', '2,500', '5.5 Baths', '1.0', '5 Baths', '1.5', '4', '~', '2.5', '4,000', '3.5 Baths', '2,000', '3,000', '8.0', '1 ba', '3.5 ba', '0', '5.0', '1,500', '7.0', '1,250', '9 Baths', '2,250', '6.0', '12 Baths', '5.5', '3,500', '1.5 Baths', '2,750', 'Bathrooms: 6', '4.5 Baths', '750', '5.5+', '6', '10 Baths', '6 ba', 'Bathrooms: 19', '10.0', '1.5 ba', '4 ba', '12 ba', '2.5+', '8', '7.5+', 'Bathrooms: 10', '0 / 0', 'Sq. Ft. ', '5 ba', '4.5+', '18 Baths', '-- baths', 'Bathrooms: 7', '7', '18', '3.5+', '1.5+', '11 Baths', '5,000', '1.75 Baths', '9', '12.0', '1.1 Baths', '6.5', 'Bathrooms: 8', '10', '19

In [23]:
# Удаляем все буквы, пробелы и двоеточия
df.baths = df.baths.str.replace('[a-zA-Z:]','', regex=True)
# Удаляем все пробелы
df.baths = df.baths.str.replace(' ','', regex=True)

In [24]:
# выполним проверку
missing_values_count = df.baths.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.baths.nunique()}\n"
      f"Уникальные значения: {list(df.baths.unique())}")

Количество пропущенных значений: 106260
Процент пропущенных значений: 28.18%
Количество уникальных значений: 148
Уникальные значения: ['3.5', '3', '2', '8', nan, '1,750', '4', '5', '1,000', '7', '2.0', '3.0', '1', '4.0', '2.1', '2.5', '4.5', '6', '2,500', '5.5', '1.0', '1.5', '~', '4,000', '2,000', '3,000', '8.0', '0', '5.0', '1,500', '7.0', '1,250', '9', '2,250', '6.0', '12', '3,500', '2,750', '750', '5.5+', '10', '19', '10.0', '2.5+', '7.5+', '0/0', '..', '4.5+', '18', '--', '3.5+', '1.5+', '11', '5,000', '1.75', '12.0', '1.1', '6.5', '16', '13', '14', '9.0', '8.5+', '17', '241', '16.0', '10.5+', '6.5+', '7,500+', '7.5', '11.0', '20', '40', '1-0/1-0/1', '14.0', '2.75', '24', '22.0', '0.00', '2.25', '3.1', '3.25', '—', '22', '32', '18.0', '0.0', '27', '26', '20.0', '9.5', '8.5', '15', '35', '3.75', '29', '116/116/116', '21', '5.2', '14.5+', '76', '13.0', '', '3-1/2-2', '15.0', '23', '3.3', '43', '1.25', '34', '2.2', '1/1/1/1', '55.0', '25', '12.5+', '44', '0.5', '4.25', '4.75', '0.75'

In [25]:
# сделаем копию что бы не перезапускать весь код в слкчае ошибок
df1=df.copy()

пропуски заполним нулями так как большинство пропущенных значений соответствуют земле где нет здания

In [26]:
# поправим кривые значения
df1['baths'] = df1['baths'].str.replace('[a-zA-Z+:]','', regex=True)
df1['baths'].replace(['nan','', '~', '..', '--', '—','0.0','0.00','0/0'], 0, inplace=True)
df1['baths'].replace(['1-2','1/1-0/1-0/1-0', '1/1/1/1','1-0/1-0/1', ], 1, inplace=True)
df1['baths'].replace(['2-1/2-1/1-1/1-1','3-1/2-2',], 2, inplace=True)
df1['baths'].replace(['116/116/116'], 116, inplace=True)
df1['baths'].replace(['7,500'], 7.5, inplace=True)
df1['baths'].replace(['5,000'], 5, inplace=True)
df1['baths'].replace(['3,500'], 3.5, inplace=True)
df1['baths'].replace(['2,750'], 2.75, inplace=True)
df1['baths'].replace(['2,250'], 2.25, inplace=True)
df1['baths'].replace(['1,250'], 1.25, inplace=True)
df1['baths'].replace(['1,500'], 1.5, inplace=True)
df1['baths'].replace(['4,000'], 4, inplace=True)
df1['baths'].replace(['2,000'], 2, inplace=True)
df1['baths'].replace(['3,000'], 3, inplace=True)
df1['baths'].replace(['2,500'], 2.5, inplace=True)
df1['baths'].replace(['1,000'], 1, inplace=True)
df1['baths'].replace(['1,750'], 1.75, inplace=True)
df1['baths'] = df1['baths'].fillna(0)

In [27]:
# переведем в числовой признак и округлим до целого числа
df1['baths'] = df1['baths'].astype(float).round()

In [28]:
# Отбираем строки по заданным условиям при помощи логических операций (& - "и", | - "или", ~ - "не")
selected_rows = df1[(df1['baths'] == 0) & ((df1['Type'] == 'other') | (df1['Type'] == 'land') | (df1['Type'] == 'ranch'))]

# Выводим полученные строки
display(selected_rows)
# Находим процент выбранных строк от общего числа строк в исходном датафрейме
percent_selected = len(selected_rows) / len(df1) * 100
# Выводим полученные строки и процент выбранных строк
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type
4,For Sale,1524 Kiscoe St,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",False,land
6,Active,552 Casanova Ct,0.0,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,"181,500",False,other
7,Active,6094 Mingle Dr,0.0,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,"68,000",False,other
11,Active,4222 False Cypress Ln,0.0,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",NaN,Houston,"[{'rating': ['6', '3', '6', '1', '5'], 'data':...","3,130",77068,NaN,TX,2,"260,000",False,other
40,Active,1104 Rotonda Cir,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,ROTONDA WEST,"[{'rating': ['6', '6', '7'], 'data': {'Distanc...",NaN,33947,NaN,FL,NaN,"54,000",False,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377003,Active,1212 Laurel St Apt 401,0.0,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Nashville,"[{'rating': ['1', '3', '2', '8', '3', '1'], 'd...",908,37203,NaN,TN,NaN,"530,000",False,other
377006,Active,1022 Lyndon Station Dr,0.0,"{'atAGlanceFacts': [{'factValue': '2020', 'fac...",NaN,Pineville,"[{'rating': ['8', '4', '7'], 'data': {'Distanc...","2,058",28134,NaN,NC,NaN,"342,969",False,other
377013,Active,2104 Russell Ave,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Charlotte,"[{'rating': ['2', '3', '3'], 'data': {'Distanc...",NaN,28216,NaN,NC,NaN,"99,990",False,other
377020,Active,Homestead Dr Lot 1,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Columbiana,"[{'rating': ['8', '9', '8', 'NR'], 'data': {'D...",0,44408,NaN,OH,NaN,"$45,000",False,land


Процент выбранных строк от общего числа строк: 19.36%


In [29]:
selected_rows = df1[(df1['baths'] == 0) & ~(df1['Type'].isin(['other', 'land', 'ranch']))]
display(selected_rows)
# Находим уникальные значения признака status в выбранных строках
unique_statuses = selected_rows['Type'].unique()
# Выводим полученные строки и процент выбранных строк
print(f"Уникальные значения признака Type в выбранных строках: {unique_statuses}")
percent_selected = len(selected_rows) / len(df1) * 100
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")


,status,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type
5,For Sale,1624 S Newkirk St,0.0,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,"$209,000",False,townhouse
15,For Sale,930 52nd St,0.0,"{'atAGlanceFacts': [{'factValue': '1905', 'fac...",NaN,Brooklyn,"[{'rating': ['8', '8', '3'], 'data': {'Distanc...",NaN,11219,NaN,NY,NaN,"1,650,000",False,single_family_home
17,Active,24 W 126th St,0.0,"{'atAGlanceFacts': [{'factValue': '1899', 'fac...",NaN,New York,"[{'rating': ['2', '5', '1', '2'], 'data': {'Di...","3,325",10027,NaN,NY,NaN,"2,650,000",False,townhouse
35,For Sale,423 N Church St #13-A,0.0,"{'atAGlanceFacts': [{'factValue': '1981', 'fac...",NaN,Charlotte,"[{'rating': ['7/10', '4/10', '8/10'], 'data': ...",900 sqft,28202,2 Beds,NC,NaN,"$260,000",False,condo
71,For Sale,8451 24th Ave SW,0.0,"{'atAGlanceFacts': [{'factValue': '1918', 'fac...",NaN,Seattle,"[{'rating': ['6/10', '3/10', '4/10'], 'data': ...",770 sqft,98106,2 Beds,WA,1.0,"$475,000",False,single_family_home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377005,For Sale,206/208 N Randolph St,0.0,"{'atAGlanceFacts': [{'factValue': '1910', 'fac...",NaN,Indianapolis,"[{'rating': ['1/10'], 'data': {'Distance': ['0...",NaN,46201,NaN,IN,NaN,"$50,000",False,single_family_home
377023,Foreclosure,460 Ovington Ave #1I,0.0,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,Brooklyn,"[{'rating': ['7/10', '4/10', '7/10'], 'data': ...",520 sqft,11209,NaN,NY,NaN,"$246,900",False,condo
377028,For Sale,653 NE 15th Ter,0.0,"{'atAGlanceFacts': [{'factValue': '1945', 'fac...",NaN,Gainesville,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","1,740 sqft",32641,3 Beds,FL,NaN,"$110,000",False,single_family_home
377033,For Sale,1107 E Euclid Ave,0.0,"{'atAGlanceFacts': [{'factValue': '1938', 'fac...",NaN,San Antonio,"[{'rating': ['4/10', '5/10', '3/10'], 'data': ...",950 sqft,78212,2 Beds,TX,1.0,"$799,500",True,single_family_home


Уникальные значения признака Type в выбранных строках: ['townhouse' 'single_family_home' 'condo' 'apartment' 'modern'
 'miscellaneous' 'multi_family_home' 'mobile_home' 'historical']
Процент выбранных строк от общего числа строк: 10.25%


In [30]:
# выполним проверку
missing_values_count = df1.baths.isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df1.baths.nunique()}\n"
      f"Уникальные значения: {list(df1.baths.unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 49
Уникальные значения: [4.0, 3.0, 2.0, 8.0, 0.0, 5.0, 1.0, 7.0, 6.0, 9.0, 12.0, 750.0, 10.0, 19.0, 18.0, 11.0, 16.0, 13.0, 14.0, 17.0, 241.0, 20.0, 40.0, 24.0, 22.0, 32.0, 27.0, 26.0, 15.0, 35.0, 29.0, 116.0, 21.0, 76.0, 23.0, 43.0, 34.0, 55.0, 25.0, 44.0, 41.0, 36.0, 30.0, 28.0, 64.0, 39.0, 42.0, 60.0, 68.0]


Итого имеем 29.61% мы заменили значением 0 из них 19.36% это ранчо чемля и иной тип что в полне может быть правдой и 10.25% 'таунхаус' 'дом на одну семью' 'кондоминиум' 'квартира' 'современный' 'разное' 'многосемейный_дом' 'мобильный_дом' 'исторический' из них минимум половина действительно может не иметь ванной комнаты, в целом такой результат можно считать приемлемым

#### Обработаем признак 'homeFacts'

In [31]:
# выполним проверку
missing_values_count = df1.homeFacts.isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df1.homeFacts.nunique()}\n"
f"Уникальные значения: {list(df1.homeFacts.unique()[:2])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 321008
Уникальные значения: ["{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}", "{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '5828 sqft', 'factLabel': 'lotsize'}, {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]}"]


С помощью регулярных выражений выдерним из этого признака значения для Value и Label

In [32]:
label = df1.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
label[100]

["'Year built'",
 "'Remodeled year'",
 "'Heating'",
 "'Cooling'",
 "'Parking'",
 "'lotsize'",
 "'Price/sqft'"]

In [33]:
Value = df.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
Value

0         ['2019', '', 'Central A/C, Heat Pump', '', '',...
1         ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2         ['1961', '1967', 'Forced Air', 'Central', 'Att...
3         ['2006', '2006', 'Forced Air', 'Central', 'Det...
4                 ['', '', '', '', '', '10,019 sqft', None]
                                ...                        
377039    ['1990', '1990', 'Other', 'Central', '2 spaces...
377040    ['1924', '', 'Radiant', '', 'None', '', '$337/...
377041    ['1950', '1950', 'Other', '', '2', '1,600 sqft...
377042           [None, None, None, None, None, None, None]
377043    ['2019', None, 'Electric', 'Central', 'No Data...
Name: homeFacts, Length: 377042, dtype: object

In [34]:
# создадим список названий признаков, удалив лишние кавычки
list_label = ','.join(label[0]).replace("'","").split(',')
list_label

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [35]:
# Создадим эти признаки в нашем фрейме и заполним значениями
for i, val in enumerate(list_label):
    df1[val]=Value.apply(lambda x: x[i])

In [36]:
# Теперь признак homeFacts можно удалить
df1 = df1.drop('homeFacts', axis=1)

#### Далее обработаем новые признаки

посмотрим на Year built

In [37]:
#удалим лишние кавычки
df1['Year built'] = df1['Year built'].str.replace("'",'', regex=True)
#Выведем уникальные значения
df1['Year built'].sort_values().unique()

array(['', '1', '1019', '1057', '1060', '1208', '1700', '1703', '1735',
       '1740', '1750', '1780', '1788', '1790', '1794', '1795', '1796',
       '1799', '1800', '1802', '1803', '1804', '1805', '1807', '1808',
       '1809', '1810', '1811', '1812', '1815', '1816', '1817', '1818',
       '1820', '1822', '1823', '1824', '1825', '1828', '1829', '1830',
       '1831', '1832', '1834', '1835', '1836', '1840', '1842', '1843',
       '1844', '1845', '1846', '1847', '1848', '1850', '1851', '1852',
       '1853', '1854', '1855', '1856', '1857', '1858', '1859', '1860',
       '1861', '1862', '1863', '1864', '1865', '1866', '1867', '1868',
       '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876',
       '1877', '1878', '1879', '1880', '1881', '1882', '1883', '1884',
       '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892',
       '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900',
       '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908',
     

* 1019', '1057', '1060 скорее всего опечатка должно выглядеть вод так 1919','1957', '1960
* '', '1','1208', '2025', '559990649990', 'No Data' заменим на -1

In [38]:
# Заменим ковычки на no date
df1['Year built'] = df1['Year built'].str.replace('^\s*$','no date', regex=True)
# Заменим некорректные значения на no date
mask= df1['Year built'].isin(['1','1208','559990649990','2025','No Data','None'])
df1.loc[mask,'Year built'] = 'no date'
# Поправим опечатки
df1.loc[df1['Year built']=='1019','Year built'] = '1919'
df1.loc[df1['Year built']=='1057','Year built'] = '1957'
df1.loc[df1['Year built']=='1060','Year built'] = '1960'
# Выведим результат обработки
df1['Year built'].sort_values().unique()

array(['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790',
       '1794', '1795', '1796', '1799', '1800', '1802', '1803', '1804',
       '1805', '1807', '1808', '1809', '1810', '1811', '1812', '1815',
       '1816', '1817', '1818', '1820', '1822', '1823', '1824', '1825',
       '1828', '1829', '1830', '1831', '1832', '1834', '1835', '1836',
       '1840', '1842', '1843', '1844', '1845', '1846', '1847', '1848',
       '1850', '1851', '1852', '1853', '1854', '1855', '1856', '1857',
       '1858', '1859', '1860', '1861', '1862', '1863', '1864', '1865',
       '1866', '1867', '1868', '1869', '1870', '1871', '1872', '1873',
       '1874', '1875', '1876', '1877', '1878', '1879', '1880', '1881',
       '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889',
       '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897',
       '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905',
       '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913',
      

In [39]:
# Отбираем строки по заданным условиям при помощи логических операций (& - "и", | - "или", ~ - "не")
selected_rows = df1[(df1['Year built'] == 'no date') & ((df1['Type'] == 'other') | (df1['Type'] == 'land') | (df1['Type'] == 'ranch'))]

# Выводим полученные строки
display(selected_rows)
# Находим процент выбранных строк от общего числа строк в исходном датафрейме
percent_selected = len(selected_rows) / len(df1) * 100
# Выводим полученные строки и процент выбранных строк
print(f"Выбранных строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,...,target,pool_encoded,Type,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
4,For Sale,1524 Kiscoe St,0.0,NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,...,"$5,000",False,land,no date,'','','','',"'10,019 sqft'",None
26,Active,5755 SW 123 Ave,1.0,NaN,Miami,"[{'rating': ['9', '10', 'NR', '6', '3'], 'data...",NaN,33183,Baths,FL,...,"799,000",False,other,no date,'','','','','—',None
40,Active,1104 Rotonda Cir,0.0,NaN,ROTONDA WEST,"[{'rating': ['6', '6', '7'], 'data': {'Distanc...",NaN,33947,NaN,FL,...,"54,000",False,other,no date,'','','','','—',None
41,For Sale,9418 Camino Santa Fe #19,0.0,NaN,San Diego,"[{'rating': ['9/10', '9/10', '9/10'], 'data': ...",NaN,92127,NaN,CA,...,"$1,100,000",False,land,no date,'','','','','9.25 acres',None
42,Active,MLS #: STRI1057289,0.0,NaN,New York,"[{'rating': ['NR', 'NR', '9', '8', '8', '2', '...",NaN,10006,NaN,NY,...,"850,000",False,other,no date,'2003','','','','—',None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376999,For Sale,6383 Cathedral Dr,0.0,NaN,Hickory,"[{'rating': ['5/10', '5/10', '7/10'], 'data': ...",NaN,28601,NaN,NC,...,"$41,000",False,land,no date,'','','','','3.34 acres',None
377000,Active,Mantova Ct,0.0,NaN,ORLANDO,"[{'rating': ['7', '6', 'NR', 'NR', '7'], 'data...",NaN,32836,NaN,FL,...,"400,000",False,other,no date,'','','','','—',None
377013,Active,2104 Russell Ave,0.0,NaN,Charlotte,"[{'rating': ['2', '3', '3'], 'data': {'Distanc...",NaN,28216,NaN,NC,...,"99,990",False,other,no date,'','','','',"'6,970 Sq. Ft.'",None
377020,Active,Homestead Dr Lot 1,0.0,NaN,Columbiana,"[{'rating': ['8', '9', '8', 'NR'], 'data': {'D...",0,44408,NaN,OH,...,"$45,000",False,land,no date,'','','','','1',None


Выбранных строк: 44783
Процент выбранных строк от общего числа строк: 11.88%


In [40]:
selected_rows = df1[(df1['Year built'] == 'no date') & ~(df1['Type'].isin(['other', 'land', 'ranch']))]
display(selected_rows)
# Находим уникальные значения признака status в выбранных строках
unique_statuses = selected_rows['Type'].unique()
# Выводим полученные строки и процент выбранных строк
print(f"Уникальные значения признака Type в выбранных строках: {unique_statuses}")
percent_selected = len(selected_rows) / len(df1) * 100
print(f"Выбранных строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,...,target,pool_encoded,Type,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
27,For Sale,2104 Lou Ellen Ln #B,3.0,NaN,Houston,"[{'rating': ['6/10', '9/10', '3/10'], 'data': ...","2,575 sqft",77018,3 Beds,TX,...,"$499,900",False,single_family_home,no date,'','','','','3828 sqft','$194/sqft'
57,For Sale,1134 Longitude Dr,2.0,NaN,Durham,"[{'rating': ['3/10', '4/10', '3/10'], 'data': ...","1,681 sqft",27713,3 Beds,NC,...,"$259,658",False,townhouse,no date,'','','','','','$154/sqft'
59,For Sale,9027 Indigo Breeze Ct,2.0,NaN,Temple Terrace,"[{'rating': ['2/10', '3/10', '2/10'], 'data': ...","1,504 sqft",33637,3 Beds,FL,...,"$244,990",False,single_family_home,no date,'','','','','','$163/sqft'
83,For Sale,3894 Chapman Bluff Dr,3.0,NaN,Spring,"[{'rating': ['9/10', '9/10', '8/10'], 'data': ...","2,265 sqft",77386,4 Beds,TX,...,"$299,999",False,single_family_home,no date,'','','','','','$132/sqft'
91,Active,15125 Mille Fiore Blvd.,2.0,NaN,Port Charlotte,"[{'rating': ['5', '4', '6', '1', 'NR', 'NR', '...","1,702",33953,3,FL,...,"$235,900",False,single_family_home,no date,'','','','',None,'$139'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376769,For Sale,1275 Stardust St,2.0,NaN,Reno,"[{'rating': ['4/10', '7/10', '10/10'], 'data':...",NaN,89503,0,NV,...,"$59,000",False,multi_family_home,no date,None,'Forced air','Central','10 spaces',None,'No Data'
376831,For Sale,1566-3 Plan in The Villas at Del Sol,2.0,NaN,Murfreesboro,"[{'rating': ['4/10', '8/10', '7/10'], 'data': ...","1,566 sqft",37127,3 Beds,TN,...,"$231,100+",False,townhouse,no date,'','','','','','$148/sqft'
376919,For Sale,6901 Acadia Ln,2.0,NaN,North Richland Hills,"[{'rating': ['9/10', '10/10', '7/10'], 'data':...","1,925 sqft",76182,2 Beds,TX,...,"$325,000",False,townhouse,no date,'','','','','','$169/sqft'
376944,For Sale,27603 Recanto,4.0,NaN,San Antonio,"[{'rating': ['10/10', '8/10', '8/10'], 'data':...","2,754 sqft",78260,4 Beds,TX,...,"$445,981",True,single_family_home,no date,'','','','','0.57 acres','$162/sqft'


Уникальные значения признака Type в выбранных строках: ['single_family_home' 'townhouse' 'condo' 'miscellaneous'
 'multi_family_home' 'apartment' 'mobile_home' 'modern']
Выбранных строк: 18700
Процент выбранных строк от общего числа строк: 4.96%


In [41]:
# выполним проверку
missing_values_count = df1['Year built'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df1['Year built'].nunique()}\n"
f"Уникальные значения: {list(df1['Year built'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 222
Уникальные значения: ['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790', '1794', '1795', '1796', '1799', '1800', '1802', '1803', '1804', '1805', '1807', '1808', '1809', '1810', '1811', '1812', '1815', '1816', '1817', '1818', '1820', '1822', '1823', '1824', '1825', '1828', '1829', '1830', '1831', '1832', '1834', '1835', '1836', '1840', '1842', '1843', '1844', '1845', '1846', '1847', '1848', '1850', '1851', '1852', '1853', '1854', '1855', '1856', '1857', '1858', '1859', '1860', '1861', '1862', '1863', '1864', '1865', '1866', '1867', '1868', '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '1880', '1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910',

In [42]:
# посмотрим на колличество no date в %
print(f"Процент no date: {round(df1[df1['Year built']=='no date'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество no date: {df1.loc[df1['Year built']=='no date', 'Year built'].count()}")

Процент no date: 16.84%
Количество no date: 63483


Большая часть значений no date соответствуют типу земля ранчо и другое 11.88% что в полне похоже на реальность, к остальным же типам относится 4.96% no date это много что бы удалить строки но и заполнить рандомно год постройки мы не можем так что оставим как есть

#### Посмотрим на признак Remodeled year

In [43]:
#удалим лишние кавычки
df1['Remodeled year'] = df1['Remodeled year'].str.replace("'",'', regex=True)
#Выведем уникальные значения
df1['Remodeled year'].sort_values().unique()

array(['', '0', '1111', '1738', '1800', '1845', '1846', '1853', '1862',
       '1869', '1870', '1874', '1876', '1877', '1880', '1883', '1884',
       '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892',
       '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900',
       '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908',
       '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916',
       '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924',
       '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932',
       '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940',
       '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948',
       '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956',
       '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
     

In [44]:
# Заменим некорректные значения на None
mask= df1['Remodeled year'].isin(['', '0', '1111'])
df1.loc[mask,'Remodeled year'] = 'None'
#Выведем уникальные значения
display(df1['Remodeled year'].sort_values().unique())

array(['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870',
       '1874', '1876', '1877', '1880', '1883', '1884', '1885', '1886',
       '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894',
       '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902',
       '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910',
       '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918',
       '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926',
       '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934',
       '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942',
       '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950',
       '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958',
       '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966',
       '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974',
       '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
      

In [45]:
# выполним проверку
missing_values_count = df1['Remodeled year'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df1['Remodeled year'].nunique()}\n"
f"Уникальные значения: {list(df1['Remodeled year'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 152
Уникальные значения: ['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870', '1874', '1876', '1877', '1880', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',

In [46]:
# посмотрим на колличество None в %
print(f"Процент None: {round(df1[df1['Remodeled year']=='None'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество None: {df1.loc[df1['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 60.0%
Количество None: 226240


Пропусков слишком много однозначно признак на удаление

In [47]:
# удалим признак 'Remodeled year'
df1 = df1.drop('Remodeled year', axis=1)

Посмотрим на признак 'Heating'

In [48]:
display(df1['Heating'].str.lower().value_counts().head(30))

Heating
'forced air'                     134307
''                               105670
'other'                           29622
'electric'                        10216
'gas'                              9295
'heat pump'                        8851
'no data'                          8610
'central air'                      7814
'central electric'                 7112
'central'                          6247
'central, electric'                4253
'baseboard'                        3815
none                               3533
'wall'                             3301
'electric heat'                    3064
'heating system'                   2708
'forced air, heat pump'            1767
'radiant'                          1485
'central air, ceiling fan(s)'      1432
'natural gas heat'                 1383
'central furnace'                  1036
'forced air, gas'                   962
'central electric, zoned'           873
'forced air heating'                810
', gas hot air/furnace'         

Признак у казывает нам на наличие отопления, возможно его нет у участвков и мобильных домов, проверим это

In [49]:
# Рассчитываем процент для признаков land и mobile_home
land_percentage = (df1['Type'].value_counts().loc['land'] / len(df1)) * 100
mobile_home_percentage = (df1['Type'].value_counts().loc['mobile_home'] / len(df1)) * 100

#Выводим результаты
print("Процент значений 'land' и 'mobile_home': {:.2f} %".format(land_percentage + mobile_home_percentage))

Процент значений 'land' и 'mobile_home': 9.27 %


признак содержит информацию отипах отопления и имеет значительное колличество пропусков ''-105434, no data - 8415, none - 3485 = 117334 значений что составляет 31%  вычтем от сюда % мобильных домов и земли, получим примерно 21% пропусков так же на вскидку увидел что у части домов где нет признака отопление есть камин или стоимость такого дома значительно ниже чем похожий но с отоплением
принимаю рещение закодировать этот признак булевыми переменными ''-105434, no data - 8415, none - 3485 будет False а остальные True

In [50]:
# удалим лишние ковычки
df1['Heating'] = df1['Heating'].str.replace("'",'', regex=True)
df1['Heating_encoded'] = df1['Heating'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [51]:
# выполним проверку
missing_values_count = df1['Heating_encoded'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df1['Heating_encoded'].nunique()}\n"
f"Уникальные значения: {list(df1['Heating_encoded'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 2
Уникальные значения: [False, True]


In [52]:
# удалим признак 'Heating'
df1= df1.drop('Heating', axis=1)

Посмотрим на признак Cooling- охлождение скорее всего кондиционер

In [53]:
df1['Cooling'].str.lower().value_counts().head(30)

Cooling
'central'                                            158742
''                                                   120306
'central air'                                         14384
'no data'                                             10615
'has cooling'                                          9730
'none'                                                 7390
'central electric'                                     6154
'wall'                                                 4017
'central gas'                                          3573
none                                                   3533
'central heating'                                      2807
'cooling system'                                       2699
'central a/c'                                          2051
'other'                                                1840
'central a/c (electric), central heat (gas)'           1646
'central a/c (electric), central heat (electric)'      1429
'refrigeration'                 

Аналогичная ситуация и с кондиционерами более 50% пропусков скорее всего кондиционер просто отсутствует

In [54]:
# удалим лишние ковычки
df1['Cooling'] = df1['Cooling'].str.replace("'",'', regex=True)
df1['Cooling_encoded'] = df1['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [55]:
# удалим признак 'Cooling'
df1= df1.drop('Cooling', axis=1)

Посмотрим на Parking 

In [56]:
df1['Parking'].str.lower().value_counts().head(30)

Parking
''                                    171755
'attached garage'                      70748
'2 spaces'                             28061
'1 space'                              14252
'no data'                              13332
'detached garage'                      13200
'carport'                               7743
'off street'                            5279
'3 spaces'                              4724
none                                    3533
'carport, attached garage'              3025
'1'                                     2936
'4 spaces'                              2917
'2'                                     2756
'none'                                  2368
'on street'                             1707
'attached garage, detached garage'      1354
'0'                                     1114
'attached garage, carport'               993
'parking desc'                           900
'6 spaces'                               755
'detached garage, attached garage'       726
'd

Признак Parking так же имеет очень много пропусков 50% и с большой вероятностью пропуски это отсутствие парковки

In [57]:
# удалим лишние ковычки
df1['Parking'] = df1['Parking'].str.replace("'",'', regex=True)
df1['Parking_encoded'] = df1['Parking'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none','0'] else False)

In [58]:
# удалим признак 'Parking'
df1 = df1.drop('Parking', axis=1)

Посмотрим на признак lotsize

In [59]:
df1['lotsize'].str.lower().value_counts().head(30)

lotsize
''               33055
none             28346
'—'              25247
'no data'         5329
'-- sqft lot'     3819
'0.26 acres'      3140
'0.25 acres'      2722
'0.28 acres'      2614
'0.27 acres'      2473
'0.29 acres'      2351
'0.34 acres'      1953
'0.31 acres'      1720
'0.32 acres'      1582
'6,098 sqft'      1525
'0.3 acres'       1436
'0.33 acres'      1412
'7,405 sqft'      1357
'0.35 acres'      1356
'1 acre'          1347
'6,534 sqft'      1292
'0.46 acres'      1286
'4,356 sqft'      1270
'10,000 sqft'     1260
'0.36 acres'      1224
'5,227 sqft'      1175
'0.37 acres'      1172
'5,000 sqft'      1117
'8,712 sqft'      1030
'0.38 acres'       994
'3,920 sqft'       993
Name: count, dtype: int64

Пропусков более 48% думаю лучше этот признак удалить так во первых это площадь участка на котором расположен дом, а во вторых этот признак числовой заполнять его нет сысла ибо мы ухудшим предсказание

In [60]:
# удалим признак 'lotsize'
df1 = df1.drop('lotsize', axis=1)

Признак Price/sqft напрямую зависит от целевого признака. Поэтому оставлять его нельзя, иначе будет утечка данных.

In [61]:
# удалим признак 'Price/sqft'
df1= df1.drop('Price/sqft', axis=1)

Посмотрим на признак fireplace

In [62]:
df1['fireplace'].str.lower().value_counts().head(30)

fireplace
yes                          71209
1                            14543
2                             2432
not applicable                1993
fireplace                      847
3                              564
living room                    433
location                       399
wood burning                   311
gas/gas logs                   300
no                             289
fireplace yn                   287
special features               279
1 fireplace                    274
0                              271
familyrm                       246
fireplace features             239
great room                     207
wood                           206
gas logs, great room           188
ceiling fan                    186
living room, wood burning      185
4                              184
family room, wood burning      149
familyrm, gas logs             146
gas                            134
gas logs, in great room        112
gas logs                       107
gas logs, 

Будем считать что пропуски в признаке наличия камина так же указывают на его отсутствие так же закодируем его булевым значением

In [63]:
df1['fireplace_encoded'] = df1['fireplace'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no'] and not pd.isna(x) else False)

In [64]:
# удалим столбец
df1.drop('fireplace', axis=1, inplace=True)

Посмотрим на признак city

In [65]:
# Приведем названия к единому виду
df1['city'] = df1['city'].str.title()
# выполним проверку
missing_values_count = df1['city'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df1['city'].nunique()}\n"
f"Уникальные значения: {list(df1['city'].sort_values().unique())}")

Количество пропущенных значений: 32
Процент пропущенных значений: 0.01%
Количество уникальных значений: 1909
Уникальные значения: [' ', '--', 'Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights', 'Albany', 'Albion', 'Alburgh', 'Alden', 'Aledo', 'Alexandria', 'Algona', 'Alhambra', 'Allen', 'Allentown', 'Allentown City', 'Allston', 'Alta Loma', 'Altadena', 'Altamonte Springs', 'Alto', 'Alvin', 'Alviso', 'Alys Beach', 'Amanda', 'Amarillo', 'American Canyon', 'Amherst', 'Anaheim', 'Anaheim Hills', 'Anderson', 'Anderson Island', 'Anderson Township', 'Anderson Twp', 'Angel Fie', 'Angola', 'Ann Arbor', 'Annandale', 'Antelope', 'Anthony', 'Antioch', 'Apex', 'Apison', 'Apopka', 'Arcadia', 'Archdale', 'Archer', 'Arden', 'Argyle', 'Arkwright', 'Arleta', 'Arlington', 'Arlington Heights', 'Arrington', 'Arvada', 'Arverne', 'Asheville', 'Asheville City', 'Ashland City', 'Ashville', 'Astoria', 'Athens', 'Atlaanta', 'Atlanta', 'Atlantic Beach', '

In [66]:
df1['city'].str.lower().value_counts().sort_index().head(30)

city
                      25
--                     9
abilene              134
abingdon             116
accokeek               3
adams                  5
addison               25
adel                  13
adelphi                6
advance                3
akron                108
alamo heights         84
albany               420
albion                 4
alburgh               28
alden                  6
aledo                 60
alexandria           136
algona                 2
alhambra              58
allen                105
allentown            103
allentown city         2
allston                1
alta loma              7
altadena              32
altamonte springs      2
alto                   2
alvin                  6
alviso                 2
Name: count, dtype: int64

Итого на удаление у нас 66 строк

In [67]:
#сделаем копию
df2=df1.copy()

In [68]:
# Удалим приставку Сity в названии городов
df2['city'] = df2['city'].str.replace(' City', '') 
#сохраняем исходное количество строк
initial_row_count = df2.shape[0]

#удаляем строки, содержащие ' ', '--' или NaN в столбце 'city'
df2 = df2[~df2['city'].isin([' ', '--', np.nan])]

#подсчитываем количество удаленных строк
removed_row_count = initial_row_count - df2.shape[0]

print(f"Количество удаленных строк: {removed_row_count}")

Количество удаленных строк: 66


In [69]:
# выполним проверку
missing_values_count = df2['city'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df2['city'].nunique()}\n"
f"Уникальные значения: {list(df2['city'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 1897
Уникальные значения: ['Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights', 'Albany', 'Albion', 'Alburgh', 'Alden', 'Aledo', 'Alexandria', 'Algona', 'Alhambra', 'Allen', 'Allentown', 'Allston', 'Alta Loma', 'Altadena', 'Altamonte Springs', 'Alto', 'Alvin', 'Alviso', 'Alys Beach', 'Amanda', 'Amarillo', 'American Canyon', 'Amherst', 'Anaheim', 'Anaheim Hills', 'Anderson', 'Anderson Island', 'Anderson Township', 'Anderson Twp', 'Angel Fie', 'Angola', 'Ann Arbor', 'Annandale', 'Antelope', 'Anthony', 'Antioch', 'Apex', 'Apison', 'Apopka', 'Arcadia', 'Archdale', 'Archer', 'Arden', 'Argyle', 'Arkwright', 'Arleta', 'Arlington', 'Arlington Heights', 'Arrington', 'Arvada', 'Arverne', 'Asheville', 'Ashland', 'Ashville', 'Astoria', 'Athens', 'Atlaanta', 'Atlanta', 'Atlantic Beach', 'Auburn', 'Augusta', 'Aurora', 'Austin', 'Ave', 'Avent

Признак schools

In [70]:
# посморим на структуру
df2.schools[5]

"[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]"

признак содержит сведения о рейтинге школы, растоянии до школы, оценки, и наименование, оценки и наименования не дают нам никакой полезной информации а вот среднее растояние до школы а так же средний рейтинг был бы крайне полезен

In [71]:
# Рассчитаем средний рейтинг
def process_rating(rating):
    rating = rating[0].replace('/10', '')
    extracted_numbers = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(extracted_numbers) if extracted_numbers else -1
    return round(average_rating, 1)

s_rating = (
    df2.schools.str.findall(r"\brating': ([\s\S]+?), 'data\b")
    .apply(lambda x: process_rating(x))
)

# Создадим признак school_rating _mean в нашем наборе данных. Пропуски заполним -1 
df2['school_rating _mean'] = s_rating

In [72]:
#Расчитаем минимальную дистанцию
s_distance = df2.schools.str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") 
# Извлечение расстояний до школ из каждой строки
s_distance = s_distance.apply(lambda x: x[0]) 
# Получение первого найденного значения списков, полученных в результате предыдущего шага
s_distance = s_distance.str.replace('[a-zA-Z]','', regex=True) 
# Удаление всех символов алфавита из расстояний
s_distance = s_distance.str.findall(r'\b([0-9]+.[0-9]+)') 
# Извлечение чисел виде Точноечисло, включая десятичные значения
s_distance = s_distance.apply(lambda x: [float(i) for i in x]) 
# Конвертирование каждого числового значения списка в тип float
school_dist_min = s_distance.apply(lambda x: -1 if len(x)==0 else min(x)) 
# Нахождение минимального значения в каждом списке s_distance, и возвращение -1, если длина списка равна 0

#Создадим признак school_dist_min в нашем наборе данных. Пропуски заполним -1
df2['school_dist_min'] = school_dist_min

In [73]:
# теперь признак schools можно удалить
df2.drop('schools', axis=1, inplace=True)

Посмотрим признак sqft- площадь

In [74]:
# выполним проверку
missing_values_count = df2['sqft'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df2['sqft'].nunique()}\n"
f"Уникальные значения: {list(df2['sqft'].sort_values().unique())}")

Количество пропущенных значений: 40516
Процент пропущенных значений: 10.75%
Количество уникальных значений: 25396
Уникальные значения: ['--', '-- sqft', '0', '1', '1 sqft', '1,000', '1,000 sqft', '1,001', '1,001 sqft', '1,002', '1,002 sqft', '1,003', '1,003 sqft', '1,004', '1,004 sqft', '1,005', '1,005 sqft', '1,006', '1,006 sqft', '1,007', '1,007 sqft', '1,008', '1,008 sqft', '1,009', '1,009 sqft', '1,010', '1,010 sqft', '1,011', '1,011 sqft', '1,012', '1,012 sqft', '1,013', '1,013 sqft', '1,014', '1,014 sqft', '1,015', '1,015 sqft', '1,016', '1,016 sqft', '1,017', '1,017 sqft', '1,018', '1,018 sqft', '1,019', '1,019 sqft', '1,020', '1,020 sqft', '1,021', '1,021 sqft', '1,022', '1,022 sqft', '1,023', '1,023 sqft', '1,024', '1,024 sqft', '1,025', '1,025 sqft', '1,026', '1,026 sqft', '1,027', '1,027 sqft', '1,028', '1,028 sqft', '1,029', '1,029 sqft', '1,030', '1,030 sqft', '1,031', '1,031 sqft', '1,032', '1,032 sqft', '1,033', '1,033 sqft', '1,034', '1,034 sqft', '1,035', '1,035 sqft',

In [75]:
df3=df2.copy()

In [76]:
s_sqft = df3.sqft[df3.sqft.notna()]
s_sqft = s_sqft.str.findall(r"\d+")
s_sqft = s_sqft.apply(lambda x: ''.join(x))
df3.loc[df3.sqft.notna(), 'sqft']=s_sqft
# Заменю пропуски в признаке sqft на значение "0"
df3.sqft = df3.sqft.fillna(0)
# также есть пустые строки вместо значения. заменим их тоже 0
df3.loc[df3.sqft=='','sqft']=0
df3.sqft = df3.sqft.astype(int)

In [77]:
# выполним проверку
missing_values_count = df3['sqft'].isna().sum()
total_count = len(df3)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df3['sqft'].nunique()}\n"
f"Уникальные значения: {list(df3['sqft'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 9890
Уникальные значения: [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 17, 20, 22, 30, 31, 33, 35, 40, 41, 45, 46, 48, 57, 60, 64, 66, 68, 72, 80, 84, 89, 90, 96, 100, 110, 111, 112, 117, 120, 121, 122, 130, 132, 136, 140, 144, 146, 147, 150, 151, 154, 160, 169, 170, 178, 179, 180, 184, 185, 190, 192, 196, 200, 205, 214, 216, 217, 218, 220, 224, 225, 228, 230, 231, 234, 240, 245, 247, 248, 250, 252, 256, 260, 264, 266, 270, 272, 275, 276, 280, 286, 288, 289, 290, 294, 297, 299, 300, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 328, 329, 330, 332, 333, 334, 335, 336, 337, 340, 342, 345, 346, 349, 350, 351, 352, 353, 354, 355, 356, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 392, 394, 395, 396, 397, 398, 399, 400, 401, 402, 

In [78]:
# Отбираем строки по заданным условиям при помощи логических операций (& - "и", | - "или", ~ - "не")
selected_rows = df3[(df3['sqft'] == 0) & ((df3['Type'] == 'other') | (df3['Type'] == 'land') | (df3['Type'] == 'ranch'))]

# Выводим полученные строки
display(selected_rows)
# Находим процент выбранных строк от общего числа строк в исходном датафрейме
percent_selected = len(selected_rows) / len(df3) * 100
# Выводим полученные строки и процент выбранных строк
print(f"Выбранных строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,street,baths,city,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
4,For Sale,1524 Kiscoe St,0.0,Palm Bay,0,32908,NaN,FL,NaN,"$5,000",False,land,no date,False,False,False,False,4.7,3.03
7,Active,6094 Mingle Dr,0.0,Memphis,0,38115,NaN,TN,NaN,"68,000",False,other,1976,False,False,False,False,2.7,0.40
26,Active,5755 SW 123 Ave,1.0,Miami,0,33183,Baths,FL,NaN,"799,000",False,other,no date,False,False,False,False,3.8,0.90
40,Active,1104 Rotonda Cir,0.0,Rotonda West,0,33947,NaN,FL,NaN,"54,000",False,other,no date,False,False,False,False,6.3,3.10
41,For Sale,9418 Camino Santa Fe #19,0.0,San Diego,0,92127,NaN,CA,NaN,"$1,100,000",False,land,no date,False,False,False,False,9.0,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376999,For Sale,6383 Cathedral Dr,0.0,Hickory,0,28601,NaN,NC,NaN,"$41,000",False,land,no date,False,False,False,False,5.7,4.48
377000,Active,Mantova Ct,0.0,Orlando,0,32836,NaN,FL,NaN,"400,000",False,other,no date,False,False,False,False,6.7,1.30
377013,Active,2104 Russell Ave,0.0,Charlotte,0,28216,NaN,NC,NaN,"99,990",False,other,no date,False,False,False,False,2.7,0.70
377020,Active,Homestead Dr Lot 1,0.0,Columbiana,0,44408,NaN,OH,NaN,"$45,000",False,land,no date,False,False,False,False,8.3,2.00


Выбранных строк: 46145
Процент выбранных строк от общего числа строк: 12.24%


In [79]:
selected_rows = df3[(df3['sqft'] == 0) & ~(df3['Type'].isin(['other', 'land', 'ranch']))]
display(selected_rows)
# Находим уникальные значения признака status в выбранных строках
unique_statuses = selected_rows['Type'].unique()
# Выводим полученные строки и процент выбранных строк
print(f"Уникальные значения признака Type в выбранных строках: {unique_statuses}")
percent_selected = len(selected_rows) / len(df3) * 100
print(f"Выбранных строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,street,baths,city,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
15,For Sale,930 52nd St,0.0,Brooklyn,0,11219,NaN,NY,NaN,"1,650,000",False,single_family_home,1905,False,True,False,False,6.3,0.20
56,Active,2014 Eastbourne Dr,2.0,Southern Pines,0,28387,2,NC,1,"$166,500",False,condo,1998,False,False,False,False,5.6,0.90
193,Foreclosure,3058 Shire Dr,0.0,Pottstown,0,19464,4 bd,PA,NaN,"$170,933",False,single_family_home,1974,True,False,False,False,4.7,2.20
221,For Sale,895 Park Ave #6/7A,8.0,New York,0,10075,6 Beds,NY,NaN,"$15,000,000",False,condo,1930,False,False,False,False,4.3,0.19
232,For Sale,545 N Dearborn St #2511,2.0,Chicago,0,60654,2 Beds,IL,2.0,"$525,000",False,condo,2003,True,True,True,False,4.0,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376850,For Sale,4513 SW 9th Ave,4.0,Cape Coral,0,33914,6 Beds,FL,NaN,"$370,000",False,multi_family_home,2019,True,True,False,False,5.5,1.51
376858,Other,9121 SW 69th Ct,5.0,Pinecrest,0,33156,5,FL,NaN,"$2,895,000",True,single_family_home,2016,True,True,True,False,2.3,1.70
376937,Active,Undisclosed Address,0.0,North Miami,0,33161,NaN,FL,NaN,"350,000",False,single_family_home,1950,True,True,False,False,2.2,0.40
376974,For Sale,2434 N Linden Pl #2,2.0,Chicago,0,60647,2 Beds,IL,2.0,"$355,000",False,condo,1906,True,True,True,False,4.0,0.17


Уникальные значения признака Type в выбранных строках: ['single_family_home' 'condo' 'miscellaneous' 'multi_family_home'
 'townhouse' 'mobile_home' 'modern' 'apartment']
Выбранных строк: 6981
Процент выбранных строк от общего числа строк: 1.85%


Большая часть 0 это земля ранчо и другое к признакам точно определяющих строение относится 1.85% нулей так что оставим как есть

Посмотрим на признак zipcode

In [80]:
# выполним проверку
missing_values_count = df3['zipcode'].isna().sum()
total_count = len(df3)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df3['zipcode'].nunique()}\n"
f"Уникальные значения: {list(df3['zipcode'].sort_values().unique())}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 4549
Уникальные значения: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02126', '02127', '02128', '02129', '02130', '02131', '02132', '02134', '02135', '02136', '02199', '02210', '02215', '02467', '04666', '04901', '04937', '04957', '04963', '05345', '05441', '05441-4400', '05443', '05444', '05445', '05448', '05450', '05454', '05455', '05456', '05457', '05461', '05462', '05464', '05469', '05472', '05473', '05478', '05482', '05483', '05487', '05488', '05491', '05642', '07004', '07004-1106', '07004-1300', '07004-1413', '07004-1528', '07004-1607', '07004-1809', '07004-1878', '07004-1935', '07004-2116', '07004-2137', '07004-2214', '08701', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010', '10011', '10012', '10013', '10014', '10016', '10017', '10018'

In [81]:
#Выведем строки с нерорректными значениями 
df3[(df3['zipcode']=='--') | (df3['zipcode']=='0')| (df3['zipcode']=='00000') ]

,status,street,baths,city,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
30258,Active,Gates Canyon Rd,0.0,Vacaville,0,0,NaN,CA,NaN,"$380,000",False,land,no date,False,False,False,False,3.7,6.50
83514,For Sale,Cornejo Ricardo Descalzi,0.0,Quito Ecuador,0,0,NaN,NY,NaN,"470,000",False,other,1995,False,True,False,False,-1.0,-1.00
231230,For Sale,0 N Gopher Canyon Rd,0.0,Bonsall,0,--,449 acres,CA,NaN,"$60,000,000",False,other,no date,False,False,False,False,5.7,2.90
235154,Other,1744 N Dixie Hwy # 1744,3.0,Fort Lauderdale,2043,--,3,FL,NaN,"$425,000",False,townhouse,2010,True,True,True,False,5.0,0.50
305481,For Sale,000 U.S. Hwy 359,0.0,Laredo,243849,0,NaN,TX,NaN,"$1,740,000",False,land,no date,False,False,False,False,4.0,7.45
308135,Active,435 pescador,0.0,Foreign Country,1100,00000,NaN,OS,NaN,"125,000",False,other,1972,False,False,False,False,-1.0,-1.00


Всего 6 строк с некорректрными значениями удалим их

In [82]:
# Отфильтровываем индексы строк, удовлетворяющих условию
indexes_to_drop = df3[(df3['zipcode']=='--') | (df3['zipcode']=='0')| (df3['zipcode']=='00000')].index

# Удаляем строки с отфильтрованными индексами
df3 = df3.drop(indexes_to_drop)

In [83]:
# посмотрим что получилось
df3.zipcode.sort_values().unique()[:100]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119', '02120', '02121', '02122', '02124',
       '02125', '02126', '02127', '02128', '02129', '02130', '02131',
       '02132', '02134', '02135', '02136', '02199', '02210', '02215',
       '02467', '04666', '04901', '04937', '04957', '04963', '05345',
       '05441', '05441-4400', '05443', '05444', '05445', '05448', '05450',
       '05454', '05455', '05456', '05457', '05461', '05462', '05464',
       '05469', '05472', '05473', '05478', '05482', '05483', '05487',
       '05488', '05491', '05642', '07004', '07004-1106', '07004-1300',
       '07004-1413', '07004-1528', '07004-1607', '07004-1809',
       '07004-1878', '07004-1935', '07004-2116', '07004-2137',
       '07004-2214', '08701', '10001', '10002', '10003', '10004', '10005',
       '10006', '10007', '10009', '10010', '10011', '10012', '10013',
       '10014', '10016', '10017', '10018', '10019', '10021', '10022',
       '10023', '10024'

In [84]:
# Поправим не корректные zipcode отбросив 2ю часть
df3.zipcode = df3.zipcode.str.replace("-.+",'', regex=True)
# посмотрим что получилось
df3.zipcode.sort_values().unique()[:100]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119', '02120', '02121', '02122', '02124',
       '02125', '02126', '02127', '02128', '02129', '02130', '02131',
       '02132', '02134', '02135', '02136', '02199', '02210', '02215',
       '02467', '04666', '04901', '04937', '04957', '04963', '05345',
       '05441', '05443', '05444', '05445', '05448', '05450', '05454',
       '05455', '05456', '05457', '05461', '05462', '05464', '05469',
       '05472', '05473', '05478', '05482', '05483', '05487', '05488',
       '05491', '05642', '07004', '08701', '10001', '10002', '10003',
       '10004', '10005', '10006', '10007', '10009', '10010', '10011',
       '10012', '10013', '10014', '10016', '10017', '10018', '10019',
       '10021', '10022', '10023', '10024', '10025', '10026', '10027',
       '10028', '10029', '10030', '10031', '10032', '10033', '10034',
       '10035', '10036', '10037', '10038', '10039', '10040', '10044',
       '10065', '100

Посмотрим на признак beds - колличество спален

In [85]:
# выполним проверку
missing_values_count = df3['beds'].isna().sum()
total_count = len(df3)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df3['beds'].nunique()}\n"
f"Уникальные значения: {list(df3['beds'].sort_values().unique())}")

Количество пропущенных значений: 91155
Процент пропущенных значений: 24.18%
Количество уникальных значений: 1181
Уникальные значения: [' ', '# Bedrooms 1st Floor', '-- bd', '-- sqft', '0', '0.0', '0.25 acres', '0.26 acres', '0.27 acres', '0.28 acres', '0.29 acres', '0.3 acres', '0.31 acres', '0.32 acres', '0.33 acres', '0.34 acres', '0.35 acres', '0.36 acres', '0.37 acres', '0.38 acres', '0.39 acres', '0.4 acres', '0.41 acres', '0.42 acres', '0.43 acres', '0.44 acres', '0.45 acres', '0.46 acres', '0.47 acres', '0.48 acres', '0.49 acres', '0.5 acres', '0.51 acres', '0.52 acres', '0.53 acres', '0.54 acres', '0.55 acres', '0.56 acres', '0.57 acres', '0.58 acres', '0.59 acres', '0.6 acres', '0.61 acres', '0.62 acres', '0.63 acres', '0.64 acres', '0.65 acres', '0.66 acres', '0.67 acres', '0.68 acres', '0.69 acres', '0.7 acres', '0.71 acres', '0.72 acres', '0.73 acres', '0.74 acres', '0.75 acres', '0.76 acres', '0.77 acres', '0.78 acres', '0.79 acres', '0.8 acres', '0.81 acres', '0.82 acres'

чесно говоря эти данные вообще не пригодны так как часть строк заполнена текстом, часть колличеством спален, а часть площадью Удаляем его

In [86]:
df3 = df3.drop('beds', axis=1)

#### Посмотрим на признак state

In [87]:
print(f"Количество пропущенных значений: {df3.state.isna().sum()}\n"
      f"Количество уникальных значений: {df3.state.nunique()}\n"
      f"Уникальные значения: {list(df3.state.unique())}")

Количество пропущенных значений: 0
Количество уникальных значений: 37
Уникальные значения: ['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR', 'DC', 'NV', 'AZ', 'GA', 'IL', 'NJ', 'MA', 'OH', 'IN', 'UT', 'MI', 'VT', 'MD', 'CO', 'VA', 'KY', 'MO', 'WI', 'ME', 'MS', 'OK', 'SC', 'MT', 'DE', 'Fl', 'AL', 'OT']


Тут все хорошо

### Теперь посмотрим на stories этажи

In [88]:
# выполним проверку
missing_values_count = df3['stories'].isna().sum()
total_count = len(df3)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df3['stories'].nunique()}\n"
f"Уникальные значения: {list(df3['stories'].sort_values().unique())}")

Количество пропущенных значений: 150515
Процент пропущенных значений: 39.93%
Количество уникальных значений: 346
Уникальные значения: [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level', '1 Level, 2 Level', '1 Level, Condo', '1 Level, Condo, Site Built', '1 Level, Non-Site Built', '1 Level, Site Built', '1 Level, Site Built, Townhouse', '1 Level, Split Foyer', '1 Level, Townhouse', '1 Leveland + Loft', '1 Story', '1 Story Basement', '1 Story, 2 Story', '1 Story, Hillside', '1 Story, Multi Level', '1 Story, Split Level', '1 Story, Townhouse', '1 Story/F.R.O.G.', '1 Story/Ranch', '1, 1', '1-2 Stories', '1-2 Stories, 3-4 Stories', '1.0', '1.00', '1.000', '1.0000', '1.2', '1.30', '1.3000', '1.5', '1.5 Level', '1.5 Level, Site Built', '1.5 Level, Site Built, Tri-Level', '1.5 Stories', '1.5 Story', '1.5 Story/Basement', '1.5, 2', '1.50', '1.500', '1.5000', '1.7', '1.70', '1.7000', '1.75', '10', '10.0', '1002.0', '11', '11.0', '11.00', '1120.0

In [89]:
# посмотрим в каких типах пропуски
df3[df3.stories.isna()].Type.value_counts()

Type
single_family_home    46300
other                 37000
land                  28815
condo                 24355
multi_family_home      5212
townhouse              3937
mobile_home            1499
ranch                  1389
modern                 1129
apartment               824
historical               44
miscellaneous            11
Name: count, dtype: int64

Пропуски встречаются как в типе земли так и в типах домов в большом колличестве заполнять рандомно не вижу смысла лучше удалить этот признак

In [90]:
df3 = df3.drop('stories', axis=1)

### Теперь посмотрим целевой признак target

In [91]:
# выполним проверку
missing_values_count = df3['target'].isna().sum()
total_count = len(df3)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df3['target'].nunique()}\n"
f"Уникальные значения: {list(df3['target'].sort_values().unique())}")

Количество пропущенных значений: 2475
Процент пропущенных значений: 0.66%
Количество уникальных значений: 43936
Уникальные значения: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713', '$1,001,990+', '$1,001,995', '$1,001,995+', '$1,002,500', '$1,002,562', '$1,002,695', '$1,002,990+', '$1,004,047', '$1,004,500', '$1,004,993', '$1,005,000', '$1,005,000+', '$1,005,187', '$1,005,803', '$1,006,033', '$1,006,118', '$1,006,302', '$1,006,419', '$1,006,500', '$1,006,635+', '$1,006,990', '$1,006,990+', '$1,007,000', '$1,007,112', '$1,007,530', '$1,008,000', '$1,008,098', '$1,008,375', '$1,008,800', '$1,009,000', '$1,009,034', '$1,009,995', '$1,010,000', '$1,010,214', '$1,010,990+', '$1,011,000', '$1,011,097', '$1,011,346', '$1,012,600', '$1,013,000', '$1,013,239', '$1,013,888', '$1,013,990', '$1,013,990+', '$1,014,311', '$1,014,900', '$1,014,925', '$1,014,995', '$1,014,995+', '$1,015,000', '$1,015,000+', '$1,015,554', 

In [92]:
# Удалим строки с пропусками так как мы не можем их заполнить и потомиспользовать
df3 = df3.dropna(subset=['target'])

В признаке есть значения указывающие что это плата за месяц с приставкой /мо а следовательно это аренда посморим на них

In [93]:
df3[df3.target.str.contains('/mo',regex=True)]

,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
547,For Rent,4323 N Central Park Ave,4.0,Chicago,3300,60618,IL,"$5,500/mo",False,single_family_home,1913,True,False,True,True,2.3,0.13
609,For Rent,220 Boylston St #1412,2.0,Boston,1673,2116,MA,"$10,500/mo",False,multi_family_home,1985,False,False,False,True,-1.0,-1.00
2075,For Rent,2830 NE 56th Ct,4.0,Fort Lauderdale,2400,33308,FL,"$6,390/mo",True,single_family_home,1965,False,False,False,False,4.0,1.19
3025,For Rent,411 Kline Aly,2.0,Clarksville,1280,37040,TN,"$1,200/mo",False,multi_family_home,2014,True,False,False,False,8.0,0.68
3645,For Rent,240 E Illinois St #2011,2.0,Chicago,1473,60611,IL,"$3,600/mo",True,multi_family_home,2003,False,False,True,False,5.5,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371654,For Rent,9436 Turrentine Dr,2.0,El Paso,1050,79925,TX,$890/mo,False,multi_family_home,no date,True,False,False,False,6.0,1.01
372321,For Rent,34 Jonquil Pl,2.0,The Woodlands,2601,77375,TX,"$2,500/mo",False,townhouse,2014,True,False,True,False,7.0,4.74
374149,For Rent,8864 Devonshire Dr,2.0,Fort Worth,2000,76131,TX,"$2,000/mo",False,single_family_home,2016,False,False,False,True,5.3,1.59
375410,For Rent,2217 W Seybert St,0.0,Philadelphia,720,19121,PA,"$1,500/mo",False,townhouse,1920,True,False,False,False,2.0,0.34


In [94]:
# вычисляем количество строк, удовлетворяющих условию
matching_rows = df3[df3.target.str.contains('/mo',regex=True)]
# вычисляем общее число строк в DataFrame
total_rows = len(df3)
# вычисляем процент строк, удовлетворяющих условию
percent_matching = (len(matching_rows) / total_rows) * 100
print('{}% строк содержат подстроку "/mo" в столбце "target".'.format(round(percent_matching, 2)))

0.11% строк содержат подстроку "/mo" в столбце "target".


Поскольку у нас стоит задача спрогнозировать именно стоимость Недвижимости а не стоимость аренды и принимая во внимание что таких строк у нас всего 0.11% полагаю что данные строки лучше удалить так как они будут давать погрешность

In [95]:
df3 = df3[~df3['target'].str.contains('/mo', regex=True)]

In [96]:
#Теперь удалим все символы кроме цыфр и переведем в числовой формат
df3.loc[:,'target'] = df3.loc[:,'target'].str.replace('[^0-9]','',regex=True)
df3.target = df3.target.astype(int)

In [97]:
display(df3[df3['target'] == df3['target']. min ()])
display(df3[df3['target'] == df3['target']. max ()])

,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
15533,Active,1230 Old Dickerson Rd,2.0,Goodlettsville,1596,37072,TN,1,False,single_family_home,1940,True,False,True,False,2.3,0.20
25334,Other,3205 N High School Rd,1.0,Indianapolis,1586,46224,IN,1,False,single_family_home,1957,True,True,True,True,1.5,0.20
84943,Foreclosure,7804 Hamilton Ave,0.0,Cincinnati,2536,45231,OH,1,False,land,1865,False,True,True,True,2.0,0.50
148219,Active,Cummings St,0.0,Huntley,0,60142,IL,1,False,land,no date,False,False,False,False,-1.0,-1.00
205315,Active,Route 47 Hwy,0.0,Huntley,0,60142,IL,1,False,land,no date,False,False,False,False,7.8,0.90
252233,Foreclosure,8136 Bradyville Pike,0.0,Murfreesboro,0,37127,TN,1,False,land,2005,False,False,False,False,6.3,3.15
252560,Active,114 S Washington St,0.0,Circleville,1771,43113,OH,1,False,single_family_home,1920,False,True,False,True,4.7,1.00
256122,Other,9115 Sea Oats Dr,3.0,Indianapolis,1780,46250,IN,1,False,condo,1989,True,True,True,True,3.7,1.90
270974,Active,Middleground Rd SW,0.0,Pataskala,0,43062,OH,1,False,land,no date,False,False,False,False,6.8,1.80
277491,Foreclosure,2902 W McMicken Ave,3.0,Cincinnati,3226,45225,OH,1,False,multi_family_home,1870,True,True,True,True,2.5,0.59


,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
132407,For Sale,875 Nimes Rd,18.0,Los Angeles,0,90077,CA,195000000,True,single_family_home,1935,True,True,True,True,2.7,1.3


По максимальной цене вопросов нет это огромная вилла а вот по минимальным есть вопросы, видим наличие странных показателей по минимальной стоимости посморим сколько вообще у нас таких аномальных значений за границу возьмем 1000

In [98]:
# считаем количество строк
count = len(df3[df3['target'] < 1000])
# считаем процент от общего числа строк
percentage = count / len(df3) * 100
# выводим результаты
print(f"Количество строк: {count}")
print(f"Процент строк: {percentage:.2f}%")

Количество строк: 77
Процент строк: 0.02%


Видим что таких строк всего 77 считаю целесообразным удалить их

In [99]:
df3 = df3.drop(df3[df3['target'] < 1000].index)

In [100]:
display(df3[df3['target'] == df3['target']. min ()])

,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
1357,Foreclosure,3314 Whitney St,2.0,Detroit,2544,48206,MI,1000,False,single_family_home,1914,True,False,True,False,3.2,0.79
1682,Other,18997 Annchester Rd,2.0,Detroit,1572,48219,MI,1000,False,single_family_home,1940,True,True,True,False,1.0,0.70
2282,Foreclosure,13715 Linnhurst St,2.0,Detroit,2048,48205,MI,1000,False,single_family_home,1925,True,False,True,False,2.9,1.08
2644,Foreclosure,14903 Sussex St,2.0,Detroit,1827,48227,MI,1000,False,single_family_home,1928,True,False,True,False,2.9,0.56
2673,Active,0 Simpson Ave,0.0,Memphis,0,38106,TN,1000,False,other,no date,False,False,False,False,2.8,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374386,Foreclosure,126 W Montana St,0.0,Detroit,1200,48203,MI,1000,False,single_family_home,1919,True,False,True,False,3.0,0.63
374953,Foreclosure,14286 Ardmore St,0.0,Detroit,960,48227,MI,1000,False,single_family_home,1925,True,False,True,True,3.1,1.01
375380,Foreclosure,4801 Garland St,0.0,Detroit,1373,48214,MI,1000,False,single_family_home,1919,True,False,False,False,3.2,1.04
376147,Foreclosure,12274 Steel St,2.0,Detroit,1258,48227,MI,1000,False,single_family_home,1928,True,False,True,False,3.1,1.06


In [101]:
missing_data = df3.isnull().sum()
missing_columns = missing_data[missing_data > 0]

if missing_columns.empty:
    print('Пропуски отсутствуют')
else:
    for column_name in missing_columns.index:
        missing_count = missing_columns[column_name]
        percentage = (missing_count / len(df3)) * 100
        print(f'{column_name}: {missing_count} пропусков ({percentage:.2f}% от общего числа данных в столбце)')


Пропуски отсутствуют


In [102]:
# Проверим дубликаты 
df3.duplicated().sum() 

303

In [103]:
# Удалим дубликаты
df3 = df3.drop_duplicates(ignore_index=True)

Признак street удалим после очистки от выбросов


отчистим наши данные от строк содержащие большое колличество пропусков 

In [118]:
result = df3[
    (df3['baths'] == 0) & 
    (df3['sqft'] == 0) & 
    (df3['Year built'] == 'no date')
]
display(result)

num_of_rows_with_nans = len(result)
total_num_of_rows = len(df3)
percentage_of_rows_with_nans = num_of_rows_with_nans / total_num_of_rows * 100

print(f"Найдено {num_of_rows_with_nans} строк с пропусками из {total_num_of_rows} общего числа строк ({percentage_of_rows_with_nans:.2f}%)")


,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
4,For Sale,1524 Kiscoe St,0.0,Palm Bay,0,32908,FL,5000,False,land,no date,False,False,False,False,4.7,3.03
40,Active,1104 Rotonda Cir,0.0,Rotonda West,0,33947,FL,54000,False,other,no date,False,False,False,False,6.3,3.10
41,For Sale,9418 Camino Santa Fe #19,0.0,San Diego,0,92127,CA,1100000,False,land,no date,False,False,False,False,9.0,0.97
42,Active,MLS #: STRI1057289,0.0,New York,0,10006,NY,850000,False,other,no date,False,False,False,False,7.0,0.10
48,Active,18b Sitton-Mills Pl,0.0,Signal Mountain,0,37377,TN,125000,False,other,no date,False,False,False,False,6.3,3.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373672,For Sale,6383 Cathedral Dr,0.0,Hickory,0,28601,NC,41000,False,land,no date,False,False,False,False,5.7,4.48
373673,Active,Mantova Ct,0.0,Orlando,0,32836,FL,400000,False,other,no date,False,False,False,False,6.7,1.30
373686,Active,2104 Russell Ave,0.0,Charlotte,0,28216,NC,99990,False,other,no date,False,False,False,False,2.7,0.70
373693,Active,Homestead Dr Lot 1,0.0,Columbiana,0,44408,OH,45000,False,land,no date,False,False,False,False,8.3,2.00


Найдено 41873 строк с пропусками из 373717 общего числа строк (11.20%)


Как мы видим у нас 11.2% строк не несущие никакой полезной информации для прогноза, удалим эти строки

In [120]:
# Удаление строк
df3 = df3[
    ~((df3['baths'] == 0) &
      (df3['sqft'] == 0) &
      (df3['Year built'] == 'no date'))
]
display(df3.head())
df3.info()

,status,street,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,240 Heather Ln,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,12911 E Heroy Ave,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,For Sale,2005 Westridge Rd,2.0,Los Angeles,3000,90049,CA,2895000,True,single_family_home,1961,True,True,True,True,6.7,1.19
3,For Sale,4311 Livingston Ave,8.0,Dallas,6457,75205,TX,2395000,False,single_family_home,2006,True,True,True,True,5.6,0.10
5,For Sale,1624 S Newkirk St,0.0,Philadelphia,897,19145,PA,209000,False,townhouse,1920,True,True,False,False,-1.0,-1.00


<class 'pandas.core.frame.DataFrame'>
Index: 331844 entries, 0 to 373716
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               331844 non-null  object 
 1   street               331844 non-null  object 
 2   baths                331844 non-null  float64
 3   city                 331844 non-null  object 
 4   sqft                 331844 non-null  int64  
 5   zipcode              331844 non-null  object 
 6   state                331844 non-null  object 
 7   target               331844 non-null  int64  
 8   pool_encoded         331844 non-null  bool   
 9   Type                 331844 non-null  object 
 10  Year built           331844 non-null  object 
 11  Heating_encoded      331844 non-null  bool   
 12  Cooling_encoded      331844 non-null  bool   
 13  Parking_encoded      331844 non-null  bool   
 14  fireplace_encoded    331844 non-null  bool   
 15  school_rating _mean  3

In [121]:
# Сохраним наши пред обработанные данные в CSV-файл
df3.to_csv("data/cleaned_data.csv", index=False)